In [2]:
import numpy as np
from collections import defaultdict

In [38]:
with open('data/input.txt') as input_file:
    t = float(input_file.readline())
    n = int(input_file.readline())
    reactants = np.zeros(n)
    for i in range(n):
        reactants[i] += float(input_file.readline())
    m = int(input_file.readline())
    equasions =  [list() for _ in range(n)]
    for _ in range(m):
        left, right = input_file.readline()[:-1].split('>')
        k, right = right.split(maxsplit=1)
        right = right[:-2]
        k = float(k)
        from_els = []
        for member in left.split('+'):
            amount, elem = member.split()
            from_els.append((float(amount), int(elem)))
        
        delta = [k]
        for am, el in from_els:
            delta.extend([el for _ in range(int(am))])
        
        for _, elem in from_els:
            cur_delta = delta.copy()
            cur_delta[0] = - cur_delta[0]
            equasions[elem].append(cur_delta)
        
        for member in right.split('+'):
            amount, elem = member.split()
            amount, elem = float(amount), int(elem)
            cur_delta = delta.copy()
            cur_delta.append(amount)
            equasions[elem].append(cur_delta)
                

In [38]:
t = float(input())
n = int(input())
reactants = np.zeros(n)
for i in range(n):
    reactants[i] += float(input())
m = int(input())
equasions =  [list() for _ in range(n)]
for _ in range(m):
    left, right = input()[:-1].split('>')
    k, right = right.split(maxsplit=1)
    right = right[:-2]
    k = float(k)
    from_els = []
    for member in left.split('+'):
        amount, elem = member.split()
        from_els.append((float(amount), int(elem)))

    delta = [k]
    for am, el in from_els:
        delta.extend([el for _ in range(int(am))])

    for _, elem in from_els:
        cur_delta = delta.copy()
        cur_delta[0] = - cur_delta[0]
        equasions[elem].append(cur_delta)

    for member in right.split('+'):
        amount, elem = member.split()
        amount, elem = float(amount), int(elem)
        cur_delta = delta.copy()
        cur_delta.append(amount)
        equasions[elem].append(cur_delta)


In [39]:
equasions

[[[-0.05, 0], [0.3, 1, 1, 1.0]],
 [[0.05, 0, 2.0], [-0.3, 1, 1], [-0.1, 2, 1], [-0.1, 3, 1]],
 [[-0.1, 2, 1]],
 [[0.1, 2, 1, 1.0], [-0.1, 3, 1]],
 [[0.1, 3, 1, 1.0]]]

In [58]:
conc = np.array(reactants)
# conc # concentration of each of elements

# euler 
N = 30000 # Grid step
dt = t / N

#foreach step
for i in np.linspace(0,t,N):
    conc = euler(conc, equasions, dt)
print("got ", conc)
print("expected [0.07183674344492988 0.10402346010604198 0.05806655446230451 0.03156383807129276 0.010369607466402765]")

got  [0.22176986 0.23705259 0.04243324 0.03637593 0.02119082]
expected [0.07183674344492988 0.10402346010604198 0.05806655446230451 0.03156383807129276 0.010369607466402765]


In [63]:
from scipy.integrate import odeint

odeint(lambda y, t: euler(y, equasions, dt), reactants, np.linspace(0,t,N))[-1]

array([7.96964334e+20, 1.68053152e+00, 3.19612238e+20, 1.54621256e+20,
       4.42385723e+19])

In [55]:
conc = reactants.copy()
conc # concentration of each of elements

# runge 
N = 500000 # Grid step
dt = t / N

#foreach step
for i in np.linspace(0,t,N):
    conc = runge_kutta(conc, dt, equasions) 
    
print("got ", conc)
print("expected [0.07183674344492988 0.10402346010604198 0.05806655446230451 0.03156383807129276 0.010369607466402765]")

TypeError: 'float' object is not subscriptable

In [ ]:
# runge
def runge_kutta(y0, step, f):
    k1 = f(y[-1]                , 0)
    k2 = f(y[-1] + step * k1 / 2, step / 2)
    k3 = f(y[-1] + step * k2 / 2, step / 2)
    k4 = f(y[-1] + step * k3    , step)
    yi = y[-1] + step / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    
    return yi

In [31]:
def euler(y0, eq, dt):
    y1 = y0.copy()
    for i, e in enumerate(eq):
        y1[i] += dt * solve(e, y0)    
    return y1

In [32]:
def solve(eq, y0):
    s = 0
    for member in eq:
        prod = 1
        for val in member:
            if type(val) is int:
                prod *= y0[val]
            else:
                prod *= val
        s += prod
    return s

In [5]:
def solve(equations, reactants, t):
    res_reactants = reactants.copy()
    print(t)
    for i, equasion in enumerate(equasions):
        res = 0
#         print(equasion)
        for member in equasion:
            mon_res = t
            for el in member:
                if isinstance(el, int):
                    el = reactants[el]
                mon_res *= el
            res += mon_res
#             print('---', member, mon_res)
#         print('res:', res)
        res_reactants[i] += res
        if res_reactants[i] < 0:
#             print('Oops', t)
            res_reactants[i] = 0
    return res_reactants

In [6]:
def euler(y0, x, f):
    y = [y0]
    step = x[1] - x[0]
    for xi in x:
        yi = y[-1] + step * f(y[-1], xi)
        y.append(yi)
    return y

def runge_kutta(y0, x, f):
    y = [y0]
    step = x[1] - x[0]
    for xi in x:
        k1 = f(y[-1]                , xi + 0)
        k2 = f(y[-1] + step * k1 / 2, xi + step / 2)
        k3 = f(y[-1] + step * k2 / 2, xi + step / 2)
        k4 = f(y[-1] + step * k3    , xi + step)
        yi = y[-1] + step / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        y.append(yi)
    return y

def go_through(y0, x, f):
    y = [y0]
    step = x[1] - x[0]
    for xi in x:
        yi = y[-1] + step * f(y[-1], step)
        y.append(yi)
    return y

In [7]:
reactants

array([0.1, 0.1, 0.1, 0. , 0. ])

In [8]:
equasions

[[[-0.05, 0], [0.3, 1, 1, 1.0]],
 [[0.05, 0, 2.0], [-0.3, 1, 1], [-0.1, 2, 1], [-0.1, 3, 1]],
 [[-0.1, 2, 1]],
 [[0.1, 2, 1, 1.0], [-0.1, 3, 1]],
 [[0.1, 3, 1, 1.0]]]

In [9]:
print(reactants)
print(solve(equasions, reactants, 1))

[0.1 0.1 0.1 0.  0. ]
1
[0.098 0.106 0.099 0.001 0.   ]


In [10]:
my_t = 1
x = np.linspace(0, my_t, my_t * 1000)
for i, react in enumerate(reactants):
    res = go_through(reactants, x, lambda y, t: solve(equasions, y, t))

0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.00100100100

0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.00100100100

0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.00100100100

0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.001001001001001001
0.00100100100

In [11]:
np.array(res).sum(1)[:20]

array([0.3       , 0.3003003 , 0.30060091, 0.30090182, 0.30120302,
       0.30150453, 0.30180634, 0.30210845, 0.30241087, 0.30271359,
       0.30301661, 0.30331993, 0.30362356, 0.30392749, 0.30423173,
       0.30453627, 0.30484111, 0.30514626, 0.30545172, 0.30575748])

In [12]:
from scipy.integrate import odeint

In [13]:
solve(equasions, reactants, 1)

1


array([0.098, 0.106, 0.099, 0.001, 0.   ])

In [14]:
my_t = 1
x = np.linspace(0, my_t, my_t * 1 + 1)
odeint(lambda yi, t: solve(equasions, yi, t), reactants, x) # , full_output = 1

0.0
0.00012156915322726173
0.00012156915322726173
0.00024313830645452346
0.00024313830645452346
0.00891107035865082
0.00891107035865082
0.017579002410847117
0.017579002410847117
0.026246934463043412
0.026246934463043412
0.05632882892992211
0.05632882892992211
0.05035832642326281
0.05035832642326281
0.07446971838348221
0.07446971838348221
0.09858111034370161
0.09858111034370161
0.12269250230392101
0.12269250230392101
0.1709152862243598
0.1709152862243598
0.2191380701447986
0.2191380701447986
0.2673608540652374
0.2673608540652374
0.3155836379856762
0.3155836379856762
0.36380642190611495
0.36380642190611495
0.4371134316086702
0.4371134316086702
0.5104204413112254
0.5104204413112254
0.5837274510137807
0.5837274510137807
0.6570344607163359
0.6570344607163359
0.7303414704188911
0.7303414704188911
0.8036484801214463
0.8036484801214463
0.9006042425531036
0.9006042425531036
0.9975600049847608
0.9975600049847608
1.094515767416418
1.094515767416418


array([[1.00000000e-01, 1.00000000e-01, 1.00000000e-01, 0.00000000e+00,
        0.00000000e+00],
       [2.73312121e-01, 2.74405383e-01, 2.69102088e-01, 2.71238768e-03,
        1.37173428e-05]])

In [15]:
np.array(res).sum(1)[-30:]

array([0.79256857, 0.79336194, 0.7941561 , 0.79495106, 0.79574681,
       0.79654336, 0.79734071, 0.79813885, 0.7989378 , 0.79973754,
       0.80053808, 0.80133943, 0.80214158, 0.80294453, 0.80374828,
       0.80455284, 0.80535821, 0.80616438, 0.80697135, 0.80777914,
       0.80858773, 0.80939714, 0.81020735, 0.81101838, 0.81183021,
       0.81264286, 0.81345632, 0.8142706 , 0.81508569, 0.8159016 ])

In [16]:
sum(map(float,'0.07183674344492988 0.10402346010604198 0.05806655446230451 0.03156383807129276 0.010369607466402765'.split()))

0.2758602035509719